In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
aisles = pd.read_csv('../input/instacart-market-basket-analysis/aisles.csv')
departments = pd.read_csv('../input/instacart-market-basket-analysis/departments.csv')
prior_product = pd.read_csv('../input/instacart-market-basket-analysis/order_products__prior.csv')
train_product = pd.read_csv('../input/instacart-market-basket-analysis/order_products__train.csv')
orders = pd.read_csv('../input/instacart-market-basket-analysis/orders.csv')
product = pd.read_csv('../input/instacart-market-basket-analysis/products.csv')


In [ ]:
# Merge products, departments, and aisles as good

goods = pd.merge(left = pd.merge(left = product, right = departments, how ='left'), right = aisles, how = 'left')

# to retain '-' and make product names more "standard"
#goods.product_name = goods.product_name.str.replace(' ', '_').str.lower() 
print(goods.shape)
goods.head()

In [ ]:
def downcast(df):
    cols = df.dtypes.index.tolist() #numbers of col: ['d_1814', 'd_1815',
    types = df.dtypes.values.tolist() #datatype in numbers of col: [dtype('int64'), dtype('int64'), 
    #-> for every column,their datatypes are all int64
    for i in range(len(types)): 
        if 'int' in str(types[i]):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(types[i]): 
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
                
        elif types[i] == np.object: #can be dates or categories 
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

goods = downcast(goods)
orders = downcast(orders)
train_product = downcast(train_product)
prior_product = downcast(prior_product)

In [ ]:
goods.dtypes, orders.dtypes, train_product.dtypes, prior_product.dtypes

In [ ]:
train_product.head(), prior_product.head(), train_product.shape, prior_product.shape, 
#train_product and prior_product is identical. both includes product and order

In [ ]:
orders.head()

In [ ]:
orders['eval_set'].value_counts()

#Three types of evaluation set: prior, train, and test. 
#Test is for submission: 75000 datapoint
#Prior is prior_product: 3214874 datapoint.
#Train is for train_product: 75000 datapoint.

In [ ]:
order_detail = pd.merge(left = train_product, right = orders, how = 'left', on = 'order_id')

In [ ]:
train_product.shape, order_detail.shape, prior_product.shape, orders.shape

In [ ]:
order_detail

In [ ]:
order_detail.shape

## Add goods data(product, aisle, departments) inside

In [ ]:
goods.head()

In [ ]:
order_detail = pd.merge(left = order_detail, right=goods[['product_id','aisle_id','department_id']], on='product_id')
#but I prefer the actual rather than id? 
order_detail.head(), order_detail.shape

# Same procedure for prior product

In [ ]:
#add order and goods data inside

prior_order_detail = pd.merge(left = prior_product, right = orders, how = 'left', on = 'order_id')
prior_order_detail = pd.merge(left = prior_order_detail, right=goods[['product_id','aisle_id','department_id']], on='product_id')

In [ ]:
prior_order_detail.head(),prior_order_detail.shape

## Concat train_product and prior_product together

In [ ]:
prior_order_detail.shape, order_detail.shape

In [ ]:
order_detail = pd.concat([prior_order_detail, order_detail])

In [ ]:
order_detail.shape

The big dataframe creation is finished

In [ ]:
#Test dataset

test_orders = orders[orders.eval_set == 'test']
test_orders

In [ ]:
len(test_orders['order_id'].unique()), len(test_orders['user_id'].unique())

Both user and order id is unique

## Baseline creation

### Code testing

In [ ]:
#Code testing: Test out what does those codes mean

test = order_detail[:50000]
user = test.groupby('user_id')['product_id'].apply(lambda x: ' '.join([str(e) for e in set(x)])).reset_index()
user.columns = ['user_id', 'products']
user

In [ ]:
#user = test.groupby('user_id')['product_id'].apply(lambda x: ' '.join([str(e) for e in set(x)])).reset_index()


df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [4, 8, 9],[4, 8, 9],[9, 8, 9]]),
                   columns=['a', 'b', 'c'])
df2



In [ ]:
grouped_df = df2.groupby('a')

for key, item in grouped_df:
    print(grouped_df.get_group(key), "\n\n")

In [ ]:
df2 = df2.groupby('a')['b'].apply(lambda x: ' '.join([str(e) for e in set(x)])).reset_index()
df2

In [ ]:
#order_detail.user_id.isin(test_orders.user_id).value_counts(), len(test_orders.user_id)

## Baseline submission: add all ordered product

In [ ]:
# test_orders id  = submission id 
# baseline submission: if the submission_id is in order_detail, 
# assume that customers will buy all products they ordered before(user-level)

user = order_detail[(order_detail.user_id.isin(test_orders.user_id))].groupby('user_id')['product_id'].apply(lambda x: ' '.join([str(e) for e in set(x)])).reset_index()
#user: all products user bought in test_orders

user.columns = ['user_id', 'products']

submission = pd.merge(left= test_orders, 
                        right= user, 
                        how='left', 
                        on='user_id')[['order_id', 'products']]
submission

In [ ]:
submission.to_csv('Ins_1.csv', encoding='utf-8', index=False)
from IPython.display import FileLink
FileLink(r'Ins_1.csv')

# Baseline submission_2: Only add reordered products

In [ ]:
order_detail_test = order_detail[(order_detail['user_id'].isin(test_orders['user_id'])) & (order_detail['reordered'] > 0)]

In [ ]:
order_detail_test

In [ ]:
user = order_detail_test.groupby('user_id')['product_id'].apply\
(lambda x: ' '.join(str(i) for i in set(x))).reset_index()

user
user.columns = ['user_id', 'products']

In [ ]:
user


#some rows that don't have any product

In [ ]:
submission_2 = pd.merge(left = test_orders, right = user, \
                     on = 'user_id', how = 'left')[['order_id', 'products']]
# submission = pd.merge(left= test_orders, 
#                         right= order_history, 
#                         how='left', 
#            on='user_id')[['order_id', 'products']]

In [ ]:
#1103 datapoint is None 

submission_2['products'] = submission_2['products'].apply(lambda x: 'None' if type(x) == float else x)

In [ ]:
submission_2.to_csv('Ins_2.csv', encoding='utf-8', index=False)
from IPython.display import FileLink
FileLink(r'Ins_2.csv')

# Baseline Submission_3: Repeat Last Order

In [ ]:
#find out what products they order in their last order

#filter out users in testing 
order_detail_test = order_detail[(order_detail['user_id'].isin(test_orders['user_id']))]

# choose their last order(the maximun order number)
user_last_order = order_detail_test.groupby('user_id')['order_number'].max().reset_index()
user_last_order.columns = ['user_id', 'order_number']

In [ ]:
# import numpy as np
# import matplotlib.mlab as mlab
# import matplotlib.pyplot as plt

# num_bins = 20
# n, bins, patches = plt.hist(user_last_order, num_bins, facecolor='blue', alpha=0.5)


In [ ]:
# Find out when order_detail and user_last_order have a match of user_id and max_order_number 


user = order_detail_test[(order_detail_test['user_id'].isin(user_last_order['user_id'])) &\
                    (order_detail_test['order_number'].isin(user_last_order['order_number']))]

#user_2 = pd.merge(left = order_detail_test, right = user_last_order, how ='left', on = ['user_id', 'order_number']) 
#this one gets a lower score            
                    
#isin is for order_detail user_id that have repetitions, so we can select all of them

user = user.groupby('user_id')['product_id'].apply(lambda x: ' '.join(str(i) for i in set(x))).reset_index()

user.columns = ['user_id', 'products']

In [ ]:
user

In [ ]:
submission_3 = pd.merge(left = test_orders, right = user, on = 'user_id', how = 'left')[['order_id','products']]
#merge on left so will have 75000 rows instead of 73891

submission_3['products'] = submission_3['products'].apply(lambda x: 'None' if type(x) == float else x)

In [ ]:
submission_3

In [ ]:
submission_3.to_csv('Ins_3.csv', encoding='utf-8', index=False)
from IPython.display import FileLink
FileLink(r'Ins_3.csv')

# Last order but only add product that been reordered

In [ ]:
order_detail_test = order_detail[(order_detail['user_id'].isin(test_orders['user_id']))]

user_last_order = order_detail_test.groupby('user_id')['order_number'].max().reset_index()
user_last_order.columns = ['user_id', 'order_number']

In [ ]:
user = order_detail_test[(order_detail_test['user_id'].isin(user_last_order['user_id'])) &\
                    (order_detail_test['order_number'].isin(user_last_order['order_number'])) &\
                         (order_detail_test['reordered'] == 1)]


user = user.groupby('user_id')['product_id'].apply(lambda x: ' '.join(str(i) for i in set(x))).reset_index()

user.columns = ['user_id', 'products']


In [ ]:
user

In [ ]:
submission_4 = pd.merge(left = test_orders, right = user, on = 'user_id', how = 'left')[['order_id','products']]
#merge on left so will have 75000 rows instead of 73891

submission_4['products'] = submission_3['products'].apply(lambda x: 'None' if type(x) == float else x)

In [ ]:
submission_4

In [ ]:
submission_3.equals(submission_4)

In [ ]:
order_detail[(order_detail['user_id'] == 4) & (order_detail['order_number']== 3)]

In [ ]:
## Find user's last order: take user_id = 4 as an example 

user_4 = order_detail[(order_detail['user_id'] == 4)]['order_number'].max()
user_test_4 = order_history[(order_history['user_id'] == 4)]['order_number'].max()

# for key, item in user_4:
#     print(user_4.get_group(key), "\n\n\n")

In [ ]:
user_4, user_test_4

In [ ]:
order_history[(order_history['user_id'] == 12)]

In [ ]:
order_detail[(order_detail['user_id'] == 4)]

## Understand submission file

In [ ]:
submission_2

In [ ]:
test_orders[test_orders['order_id'] == 2774568]

In [ ]:
test_orders[test_orders['user_id'] == 3]

In [ ]:
order_detail[order_detail['user_id'] == 3].head(12)

## Generating Association rules practice

#### Count numbers of items 

In [ ]:
#from 
import numpy as np
from itertools import combinations, groupby
from collections import Counter

# Sample data
basket = np.array([[1,'apple'], [1,'egg'], [1,'milk'], [2,'egg'], [2,'milk']], dtype=object)

In [ ]:
#from datacamp
from itertools import permutations

transactions = [['milk', 'bread', 'biscuit'], ['bread', 'milk', 'biscuit', 'cereal'], ['bread', 'tea'], ['jam', 'bread', 'milk'], ['tea', 'biscuit']]
flattened = [i for t in transactions for i in t]
# for t in transactions, for i in t -> return i 
flattened

In [ ]:
# numbers = [[1,2,3], [1,2], [1,4], [1], [1,2,3,4]]

numbers = [12, 34, 1, 4, 4, 67, 37, 9, 0, 81]
result = [number*2 for number in numbers if number >=5]
result

## Order data investigation 

In [ ]:
orders.head()

In [ ]:
## How many orders per user order? 

user = orders.groupby('user_id').sum()
user['order_number']

In [ ]:
orders['order_id'].mean(), orders['order_id'].max(), orders['order_id'].min()

In [ ]:
#try to understand the code
order_details = pd.concat(
        [ 
             order_details,
            pd.merge(left=pd.merge(
                            left=op_prior.loc[indexes[i]:indexes[i+1], :],
                            right=goods[['product_id', 
                                         'aisle_id', 
                                         'department_id' ]], 
                            how='left',
                            on='product_id'),
                     right=orders, 
                     how='left', 
                     on='order_id'
                ) #.